In [0]:
#Loading up our drive on google colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
import json
import random
import re
import pandas as pd
from numpy import int64

In [0]:
#Cloning the gpt-2 model
!git clone https://github.com/tenoke/gpt-2

fatal: destination path 'gpt-2' already exists and is not an empty directory.


In [0]:
#changing the root directory
cd gpt-2

/content/gpt-2


In [0]:
#Installing the required files in the requirements.txt file
!pip3 install -r requirements.txt

     |████████████████████████████████| 81kB 3.8MB/s 
     |████████████████████████████████| 604kB 7.3MB/s 
     |████████████████████████████████| 51kB 22.3MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=e1eaa4ee80d3858cc93eeb1732fbafb31efd9de92265e0ce5d958f6af6a1731a
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
  Created wheel for regex: filename=regex-2017.4.5-cp36-cp36m-linux_x86_64.whl size=533173 sha256=efd7fc511e98393319c38200e565f6cb2a3c6a5bb2c68da59477477f51613995
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
#downloading the model
!python download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 601kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 40.0Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 1.00Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:25, 56.5Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 6.54Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 26.4Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 40.3Mit/s]                                                       


In [0]:
#setting up the kaggle api so that we can access our ubuntu corpus dataset
from google.colab import files
files.upload()

In [0]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 69 Oct 12 16:24 kaggle.json


In [0]:
#Installing the Kaggle API client
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d rtatman/ubuntu-dialogue-corpus

 99% 788M/799M [00:07<00:00, 104MB/s] 
100% 799M/799M [00:07<00:00, 117MB/s]


In [0]:
!unzip ubuntu-dialogue-corpus.zip

Archive:  ubuntu-dialogue-corpus.zip
  inflating: Ubuntu-dialogue-corpus/dialogueText.csv  
  inflating: Ubuntu-dialogue-corpus/dialogueText_196.csv  
  inflating: Ubuntu-dialogue-corpus/dialogueText_301.csv  
  inflating: toc.csv                 


In [0]:
!mkdir ubuntu-data ubuntu-npz

In [0]:
datasets = pd.read_csv('Ubuntu-dialogue-corpus/dialogueText_196.csv', parse_dates=['date'], chunksize=1200000)

In [0]:
i = 1

for dataset in datasets:
  dataset['date'] = dataset['date'].astype(int64) // 10**9
  
  text_corpus = ''
  current = None
  for msg in dataset.itertuples():
    if msg.dialogueID != current:
      current = msg.dialogueID
      text_corpus += '\n\n'
    try:
      text_corpus += f"({msg.date}) {msg._4}: {msg.text}\n"
    except KeyError:
      pass
  
  with open(f'ubuntu-data/ubuntu-cleaned-{i}.txt', 'w') as f:
    f.write(text_corpus)
  del(text_corpus)
  i +=1
    
del(datasets)
del(dataset)

In [0]:
!du -h ubuntu-data/*

91M	ubuntu-data/ubuntu-cleaned-1.txt
92M	ubuntu-data/ubuntu-cleaned-2.txt
90M	ubuntu-data/ubuntu-cleaned-3.txt
89M	ubuntu-data/ubuntu-cleaned-4.txt
89M	ubuntu-data/ubuntu-cleaned-5.txt
86M	ubuntu-data/ubuntu-cleaned-6.txt
85M	ubuntu-data/ubuntu-cleaned-7.txt
60M	ubuntu-data/ubuntu-cleaned-8.txt


In [0]:
!PYTHONPATH=src ./encode.py ubuntu-data/ubuntu-cleaned-1.txt ubuntu-npz/ubuntu-cleaned-1.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/ubuntu-cleaned-2.txt ubuntu-npz/ubuntu-cleaned-2.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/ubuntu-cleaned-3.txt ubuntu-npz/ubuntu-cleaned-3.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/ubuntu-cleaned-4.txt ubuntu-npz/ubuntu-cleaned-4.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/ubuntu-cleaned-5.txt ubuntu-npz/ubuntu-cleaned-5.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/ubuntu-cleaned-6.txt ubuntu-npz/ubuntu-cleaned-6.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/ubuntu-cleaned-7.txt ubuntu-npz/ubuntu-cleaned-7.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/ubuntu-cleaned-8.txt ubuntu-npz/ubuntu-cleaned-8.txt.npz --model_name 345M

Reading files
100% 1/1 [03:23<00:00, 203.31s/it]
Writing ubuntu-npz/ubuntu-cleaned-1.txt.npz
Reading files
100% 1/1 [03:26<00:00, 206.89s/it]
Writing ubuntu-npz/ubuntu-cleaned-2.txt.npz
Reading files
100% 1/1 [03:12<00:00, 192.88s/it]
Writing ubuntu-npz/ubuntu-cleaned-3.txt.npz
Reading files
100% 1/1 [03:20<00:00, 200.44s/it]
Writing ubuntu-npz/ubuntu-cleaned-4.txt.npz
Reading files
100% 1/1 [03:10<00:00, 190.23s/it]
Writing ubuntu-npz/ubuntu-cleaned-5.txt.npz
Reading files
100% 1/1 [03:08<00:00, 188.30s/it]
Writing ubuntu-npz/ubuntu-cleaned-6.txt.npz
Reading files
100% 1/1 [02:59<00:00, 179.14s/it]
Writing ubuntu-npz/ubuntu-cleaned-7.txt.npz
Reading files
100% 1/1 [02:10<00:00, 130.59s/it]
Writing ubuntu-npz/ubuntu-cleaned-8.txt.npz


In [0]:
!du -h ubuntu-npz/*

41M	ubuntu-npz/ubuntu-cleaned-1.txt.npz
46M	ubuntu-npz/ubuntu-cleaned-2.txt.npz
41M	ubuntu-npz/ubuntu-cleaned-3.txt.npz
41M	ubuntu-npz/ubuntu-cleaned-4.txt.npz
39M	ubuntu-npz/ubuntu-cleaned-5.txt.npz
39M	ubuntu-npz/ubuntu-cleaned-6.txt.npz
37M	ubuntu-npz/ubuntu-cleaned-7.txt.npz
27M	ubuntu-npz/ubuntu-cleaned-8.txt.npz


In [0]:
!cp -r ubuntu-npz/ /content/drive/My\ Drive/ubuntu-npz/

In [22]:
!PYTHONPATH=src ./train.py --dataset ubuntu-npz/ --sample_every=250 --stop_after=1501 --model_name 345M


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2019-10-12 17:05:24.539670: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-10-12 17:05:24.540134: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1615480 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-12 17:05:24.540177: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2019-10-12 17:05:24.568243: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-10-12 17:05:24.696126: I tensorflow/stream_executor/cu

In [23]:
!PYTHONPATH=src ./train.py --dataset ubuntu-npz/ --sample_every=250 --learning_rate 0.0001 --stop_after 3501 --model_name 345M


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2019-10-12 18:28:24.248411: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-10-12 18:28:24.248724: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x17d5480 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-12 18:28:24.248761: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2019-10-12 18:28:24.259384: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-10-12 18:28:24.357657: I tensorflow/stream_executor/cu

In [0]:
# clean up old saves
files = []
iterations = []
for p, d, f in os.walk('/content/gpt-2/checkpoint/'):
    for file in f:
        r = re.findall('model-(\d+).', file)
        if r:
            iterations.append(int(r[0]))
            files.append(f'{p}/{file}')


last_saved = str(max(iterations))
for f in files:
  if last_saved not in f:
    os.remove(f)

In [0]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/345M/

# **Unconditioned samples**

In [27]:
!python3 src/generate_unconditional_samples.py --top_k 40 --temperature 0.95 --model_name 345M



2019-10-12 19:05:26.807445: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-10-12 19:05:26.830474: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-10-12 19:05:26.831307: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-10-12 19:05:26.836292: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2019-10-12 19:05:26.846151: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2019-10-12 19:05:26.852067: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.1

# **QuestionAnswering**

In [28]:
!python3 src/interactive_conditional_samples.py --model_name='345M'  --nsamples=2 --top_k=100 --temperature=1



2019-10-12 19:07:01.050100: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-10-12 19:07:01.070449: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-10-12 19:07:01.071289: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-10-12 19:07:01.071577: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2019-10-12 19:07:01.073147: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2019-10-12 19:07:01.074634: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.1

# **Summarization**

In [29]:
!python3 src/interactive_conditional_samples.py --model_name='345M'  --nsamples=2 --top_k=100 --temperature=1



2019-10-12 19:08:32.666490: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-10-12 19:08:32.684602: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-10-12 19:08:32.685431: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-10-12 19:08:32.685786: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2019-10-12 19:08:32.687320: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2019-10-12 19:08:32.688850: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.1